In [1]:
# import library
import pickle
import pandas as pd
import numpy as np

import xgboost as xgb
import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto(
#     gpu_options=tf.GPUOptions(
#         visible_device_list="0", 
#         allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
#     )
# )
# set_session(tf.Session(config=config))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read data
with open('../data/HCD35_pos.pickle', mode='rb') as fp:
    df_3 = pickle.load(fp)
    
df = pd.concat([
    df_3,
    pd.read_csv('../data/decomp_pos.csv').drop('Subclass', axis=1)
], axis=1)

In [3]:
# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [4]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# result append to list
t = []
t.append(round(clf.score(X_test, y_test)*100, 2))
pickle.dump(clf, open('../model/rf_pos_HCD35+decomp.sav', 'wb'))

In [5]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

pickle.dump(gbm, open('../model/lgbm_pos_HCD35+decomp.sav', 'wb'))
# result append to list
t.append(round(gbm.score(X_test, y_test)*100, 2))

[1]	valid_0's multi_logloss: 1.28903
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.20755
[3]	valid_0's multi_logloss: 1.14199
[4]	valid_0's multi_logloss: 1.08496
[5]	valid_0's multi_logloss: 1.03621
[6]	valid_0's multi_logloss: 0.991759
[7]	valid_0's multi_logloss: 0.950848
[8]	valid_0's multi_logloss: 0.913097
[9]	valid_0's multi_logloss: 0.880727
[10]	valid_0's multi_logloss: 0.851425
[11]	valid_0's multi_logloss: 0.823601
[12]	valid_0's multi_logloss: 0.799053
[13]	valid_0's multi_logloss: 0.776542
[14]	valid_0's multi_logloss: 0.756606
[15]	valid_0's multi_logloss: 0.735461
[16]	valid_0's multi_logloss: 0.715961
[17]	valid_0's multi_logloss: 0.697946
[18]	valid_0's multi_logloss: 0.679269
[19]	valid_0's multi_logloss: 0.664019
[20]	valid_0's multi_logloss: 0.650185
[21]	valid_0's multi_logloss: 0.634642
[22]	valid_0's multi_logloss: 0.619828
[23]	valid_0's multi_logloss: 0.608332
[24]	valid_0's multi_logloss: 0.596635
[25]	valid_0's mu

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [6]:
# set parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [7]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
pickle.dump(model, open('../model/xgb_pos_HCD35+decomp.sav', 'wb'))
t.append(round(model.score(X_test, y_test)*100, 2))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [8]:
tf.set_random_seed(42)

# make keras model

inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_pos_HCD35+decomp.h5')
t.append(round(model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2))

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 3s 3ms/step - loss: 1.9419 - acc: 0.2381 - val_loss: 1.4529 - val_acc: 0.2560
Epoch 2/100
1340/1340 [==============================] - 1s 900us/step - loss: 1.7768 - acc: 0.2642 - val_loss: 1.4466 - val_acc: 0.3036
Epoch 3/100
1340/1340 [==============================] - 1s 887us/step - loss: 1.7454 - acc: 0.2657 - val_loss: 1.4603 - val_acc: 0.2976
Epoch 4/100
1340/1340 [==============================] - 1s 894us/step - loss: 1.5972 - acc: 0.3052 - val_loss: 1.4482 - val_acc: 0.2946
Epoch 5/100
1340/1340 [==============================] - 1s 887us/step - loss: 1.5957 - acc: 0.3067 - val_loss: 1.4239 - val_acc: 0.3155
Epoch 6/100
1340/1340 [==============================] - 1s 870us/step - loss: 1.5495 - acc: 0.3291 - val_loss: 1.4098 - val_acc: 0.3244
Epoch 7/100
1340/1340 [==============================] - 1s 848us/step - loss: 1.5327 - acc: 0.3276 - val_loss: 1.3543 - val_acc: 0.3

In [9]:
f = []
f.append(t)

In [2]:
with open('../data/HCD35_neg.pickle', mode='rb') as fp:
    df_3 = pickle.load(fp)
    
df = pd.concat([
    df_3,
    pd.read_csv('../data/decomp_neg.csv').drop('Subclass', axis=1)
], axis=1)

In [3]:
# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [12]:
# define and fit 
clf = rf()
clf = clf.fit(X_train, y_train)

# result append to list
t = []
t.append(round(clf.score(X_test, y_test)*100, 2))
pickle.dump(clf, open('../model/rf_ng_HCD35+decomp.sav', 'wb'))

In [13]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

pickle.dump(gbm, open('../model/lgbm_ng_HCD35+decomp.sav', 'wb'))
# result append to list
t.append(round(gbm.score(X_test, y_test)*100, 2))

[1]	valid_0's multi_logloss: 1.28919
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.20761
[3]	valid_0's multi_logloss: 1.14127
[4]	valid_0's multi_logloss: 1.08399
[5]	valid_0's multi_logloss: 1.03192
[6]	valid_0's multi_logloss: 0.985134
[7]	valid_0's multi_logloss: 0.948937
[8]	valid_0's multi_logloss: 0.919607
[9]	valid_0's multi_logloss: 0.884904
[10]	valid_0's multi_logloss: 0.857844
[11]	valid_0's multi_logloss: 0.831593
[12]	valid_0's multi_logloss: 0.811628
[13]	valid_0's multi_logloss: 0.793627
[14]	valid_0's multi_logloss: 0.77842
[15]	valid_0's multi_logloss: 0.762973
[16]	valid_0's multi_logloss: 0.746611
[17]	valid_0's multi_logloss: 0.734568
[18]	valid_0's multi_logloss: 0.72581
[19]	valid_0's multi_logloss: 0.715228
[20]	valid_0's multi_logloss: 0.706909
[21]	valid_0's multi_logloss: 0.699649
[22]	valid_0's multi_logloss: 0.692077
[23]	valid_0's multi_logloss: 0.68506
[24]	valid_0's multi_logloss: 0.678532
[25]	valid_0's multi

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [4]:
# setting parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [6]:
# define and fit 
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
t.append(round(model.score(X_test, y_test)*100, 2))
pickle.dump(model, open('../model/xgb_ng_HCD35+decomp.sav', 'wb'))

In [8]:
tf.set_random_seed(42)

# make keras model

inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_ng_HCD35+decomp.h5')
# t.append(round(model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2))

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 2s 4ms/step - loss: 1.9764 - acc: 0.2627 - val_loss: 1.2929 - val_acc: 0.4255
Epoch 2/100
373/373 [==============================] - 0s 337us/step - loss: 1.9920 - acc: 0.2574 - val_loss: 1.2970 - val_acc: 0.4787
Epoch 3/100
373/373 [==============================] - 0s 359us/step - loss: 1.8641 - acc: 0.2574 - val_loss: 1.3592 - val_acc: 0.4787
Epoch 4/100
373/373 [==============================] - 0s 322us/step - loss: 1.7246 - acc: 0.3271 - val_loss: 1.4171 - val_acc: 0.4681
Epoch 5/100
373/373 [==============================] - 0s 385us/step - loss: 1.7972 - acc: 0.3217 - val_loss: 1.4761 - val_acc: 0.4681
Epoch 6/100
373/373 [==============================] - 0s 401us/step - loss: 1.6276 - acc: 0.3807 - val_loss: 1.4736 - val_acc: 0.4681
Epoch 7/100
373/373 [==============================] - 0s 334us/step - loss: 1.5763 - acc: 0.3995 - val_loss: 1.4448 - val_acc: 0.4681
Epoch 8/100


In [ ]:
f.append(t)

In [ ]:
# save to csv
pd.DataFrame(
    f, 
    columns=['Random Forest', 'LightGBM', 'XGBoost', 'Keras'],
    index= ['Decomposition + HCD35_Positive', 'Decomposition + HCD35_Negative']
)

In [ ]:
pd.DataFrame(
    f, 
    columns=['Random Forest', 'LightGBM', 'XGBoost', 'Keras'],
    index= ['Decomposition + HCD35_Positive', 'Decomposition + HCD35_Negative']
).T.to_csv('../result/Decomposition + HCD35.csv')